In [6]:
#!pip install tensorflow-gpu
#!pip install tensorflow
#!pip install keras

In [7]:
from PIL import Image
import random
import warnings
import os
import shutil
from PIL import ImageFile

from tensorflow import keras

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import StratifiedKFold
keras.__version__

'2.7.0'

In [8]:
base_dir = r"E:\Koding\Python\Kuliah\S2\AI\Projek\Dataset"

training_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validation_dir = os.path.join(base_dir, 'Validation')

In [9]:
sourceFiles = []
classLabels = ['Pneumonia', 'Covid', 'Normal']
picture_size = (150, 150)
batch_size = 32
epoch = 10

In [10]:
def transferBetweenFolders(source, dest, splitRate):
    global sourceFiles
    sourceFiles = os.listdir(source)
    if (len(sourceFiles) != 0):
        transferFileNumbers = int(len(sourceFiles) * splitRate)
        transferIndex = random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source + str(sourceFiles[eachIndex]), dest + str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")


def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(base_dir + '/' + source + '/' + label + '/',
                                base_dir + '/' + dest + '/' + label + '/',
                               splitRate)

In [11]:
# First, check if test folder is empty or not, if not transfer all existing files to train
transferAllClassBetweenFolders('Testing', 'Training', 1.0)
transferAllClassBetweenFolders('Validation', 'Training', 1.0)
# Now, split some part of train data into the test folders.
transferAllClassBetweenFolders('Training', 'Testing', 0.20)

No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!


In [12]:
X = []
Y = []

def prepareNameWithLabels(folderName):
    sourceFiles = os.listdir(base_dir + '/Training/' + folderName)
    for val in sourceFiles:
         X.append(val)
         if (folderName == classLabels[0]):
             Y.append(0)
         elif (folderName == classLabels[1]):
             Y.append(1)
         else:
             Y.append(2)
             
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])

X = np.asarray(X)
Y = np.asarray(Y)

In [16]:
import numpy as np
from keras import layers
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
# from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

def identity_block(X, activation, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', 
               name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation(activation)(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', 
               name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation(activation)(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', 
               name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation(activation)(X)
    
    
    return X

def convolutional_block(X, activation, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation(activation)(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', 
               name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation(activation)(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', 
               name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), 
                        padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation(activation)(X)
    
    
    return X

def ResNet50(input_shape=(*picture_size, 3), classes = len(classLabels)):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Activation Function
    activation = 'relu'

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation(activation)(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = Dropout(0.4)(X)

    # Stage 2
    X = convolutional_block(X, activation, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = BatchNormalization(axis=3, name='bn_conv2')(X)
    X = identity_block(X, activation, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, activation, 3, [64, 64, 256], stage=2, block='c')
    X = Dropout(0.4)(X)

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, activation, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = BatchNormalization(axis=3, name='bn_conv3')(X)
    X = identity_block(X, activation, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, activation, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, activation, 3, [128, 128, 512], stage=3, block='d')
    X = Dropout(0.4)(X)

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, activation, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = BatchNormalization(axis=3, name='bn_conv4')(X)
    X = identity_block(X, activation, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, activation, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, activation, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, activation, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, activation, 3, [256, 256, 1024], stage=4, block='f')
    X = Dropout(0.4)(X)

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, activation, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = BatchNormalization(axis=3, name='bn_conv5')(X)
    X = identity_block(X, activation, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, activation, 3, [512, 512, 2048], stage=5, block='c')
    X = Dropout(0.4)(X)

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Dropout(0.3)(X)
    X = Flatten()(X)
    X = Dense(512, activation=activation)(X)
    X = Dense(128, activation=activation)(X)
    X = Dropout(0.3)(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    learning_rate = 0.1
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    losses = 'categorical_crossentropy'
    model.compile(optimizer=optimizer, loss=losses, metrics=['accuracy'])

    return model


c:\users\hanun\venv\lib\site-packages\keras\backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [17]:
cnn = ResNet50(input_shape = (*picture_size, 3), classes = len(classLabels))
cnn.summary()

filepath = 'Resnet.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
                                                 verbose=1, save_weights_only=False,
                                                 save_best_only=True, mode='min')

logdir = os.path.join(base_dir, 'logs/mobilenet')
tfboard = keras.callbacks.TensorBoard(log_dir=logdir)

callbacks_list = [checkpoint, tfboard]

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 156, 156, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1 (Conv2D)                 (None, 75, 75, 64)   9472        ['zero_padding2d_1[0][0]']       
                                                                                           

In [18]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
history_loss = []
history_acc = []
history_val_loss = []
history_val_acc = []
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('Validation', 'Training', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)

    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        if(Y_val[eachIndex]==0):
            classLabel=classLabels[0]
        elif(Y_val[eachIndex]==1):
            classLabel=classLabels[1]
        else:
            classLabel=classLabels[2]
        #Then, copy the validation images to the validation folder
        shutil.move(base_dir+'/Training/'+classLabel+'/'+X_val[eachIndex],
                    base_dir+'/Validation/'+classLabel+'/'+X_val[eachIndex])

    picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
    training_set = picture_datagen.flow_from_directory(training_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

    # Preprocessing the Test set
    test_set = picture_datagen.flow_from_directory(validation_dir,
                                                   shuffle=True,
                                                   target_size=picture_size,
                                                   batch_size=batch_size,
                                                   class_mode='categorical')


    # Training the CNN on the Training set and evaluating it on the Test set
    fit = cnn.fit(training_set,
                    epochs=epoch,
                    validation_data=test_set)
    
    for i in range(epoch):
      history_loss.append(fit.history['loss'][i])
      history_acc.append(fit.history['accuracy'][i])
      history_val_loss.append(fit.history['val_loss'][i])
      history_val_acc.append(fit.history['val_accuracy'][i])

Results for fold 1
Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Epoch 1/10
45/60 [=====================>........] - ETA: 3:22 - loss: 637.8075 - accuracy: 0.3708

KeyboardInterrupt: 

In [ ]:
print(fit.history['loss'][1])

In [ ]:
import pylab as pl
pl.figure()

pl.subplot(121)
pl.ylim([0,1])
pl.plot(history_acc)
pl.title('Accuracy:')
pl.plot(history_val_acc)
pl.legend(('Train', 'Val'))

pl.subplot(122)
pl.ylim([0,1])
pl.plot(history_loss)
pl.title('Cost:')
pl.plot(history_val_loss)
pl.legend(('Train', 'Val'))

In [ ]:
pl.figure()

pl.subplot(121)
pl.ylim([0,1])
pl.plot(history_acc)
pl.title('Accuracy train:')

pl.subplot(122)
pl.ylim([0,1])
pl.title('Accuracy Val:')
pl.plot(history_val_acc)

In [ ]:
pl.figure()

pl.subplot(121)
pl.plot(history_loss)
pl.title('Loss Train:')

pl.subplot(122)
pl.title('Loss Val:')
pl.plot(history_val_loss)

In [ ]:
def plot_results(results):
    pl.figure()

    pl.subplot(121)
    pl.plot(results['train_acc'])
    pl.title('Accuracy:')
    pl.plot(results['test_acc'])
    pl.legend(('Train', 'Test'))

    pl.subplot(122)
    pl.plot(results['train_loss'])
    pl.title('Cost:')
    pl.plot(results['test_loss'])
    pl.legend(('Train', 'Test'))

In [ ]:
# menghitung akurasi dari training data

picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
training_set = picture_datagen.flow_from_directory(test_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')
score = cnn.evaluate(training_set)

print('Loss: {:0.2f}%'.format(score[0] * 100))
print('Accuracy: {:0.2f}%'.format(score[1] * 100))

In [ ]:
modelpath = os.path.join(base_dir, filepath)
cnn.save(modelpath)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, confusion_matrix, roc_curve, auc, recall_score
import pylab as pl

def my_metrics(y_true, y_pred, classes, predictions):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average="macro")
    # f1Score=f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(classes, predictions, average='macro', multi_class='ovr')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("AUC : {}".format(roc_auc))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, roc_auc

In [ ]:
predictions = cnn.predict(training_set, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = training_set.classes
classes = []
for i in true_classes:
    if(i == 0):
        classes.append([1,0,0])
    elif(i == 1):
        classes.append([0,1,0])
    else:
        classes.append([0,0,1])

my_metrics(true_classes, yPredictions, classes, predictions)